In [ ]:
import sys
import os
sys.path.append("../")
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#https://github.com/szagoruyko/pytorchviz

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
import tensorboardX
print("torch:",torch.__version__)
print("tensorboardX:",tensorboardX.__version__)

In [ ]:
# CONFIGURE
data_root = 'graph_web/session_data'
session_id = "minst1"
import shutil
full_path = os.path.join(data_root,session_id)
try:
    shutil.rmtree(full_path)
except OSError as e:
    print("Error: %s : %s" % (full_path, e.strerror))
    
from modelinspector.inspector import Inspector
inspector = Inspector(session_id,data_root)
state_log_freq = 100
metric_log_freq = 100

In [ ]:
from modelinspector.inspector import save_last_state

In [ ]:
from ipywidgets import IntProgress

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        acc))
    return acc

def train(dry_run, log_interval, model, device, train_loader, test_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        i = batch_idx
        try:

            if i % metric_log_freq == 0 or i % state_log_freq == 0: 
                
                inspector.log_metrics(
                    epoch=epoch,
                    itr=i, 
                    metrics={
                        'loss':loss.item()}),
                
            if i % state_log_freq == 0:
                test_acc = test(model, device, test_loader)
                inspector.log_state(
                    epoch=epoch,
                    itr=i, 
                    model=model,
                    input_dict={'input.1':data},
                    output_dict={'output.1':output},
                    additional_info={
                        'pred': output.argmax(dim=1, keepdim=True),
                        'loss':loss,
                        'class_label':target,
                        'test_acc':test_acc})
                
#                 if test_acc > 96:
#                     break

        except Exception as e:
            print(e)
            raise e
        
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

seed = 10
batch_size = 10
test_batch_size= 100
use_cuda = torch.cuda.is_available()
epochs=1
torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.01)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(0, epochs):    
    train(False, 10000 , model, device, train_loader, test_loader, optimizer, epoch)
    scheduler.step()
